https://keras.io/keras_tuner/getting_started/#start-the-search

In [1]:
import keras
import numpy as np
from keras import layers
import keras_tuner as kt

In [2]:
(X, y), (X_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
X_train = X[:-10000]
X_val = X[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

In [8]:
X_train = np.expand_dims(X_train, -1).astype("float64") / 255.0
X_val = np.expand_dims(X_val, -1).astype("float64") / 255.0
X_test = np.expand_dims(X_test, -1).astype("float64") / 255.0

In [12]:
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [15]:
def build_model(hp):
    model = keras.Sequential()

    model.add(layers.Flatten())

    # Tune the number of layers
    for i in range(hp.Int("n_layers", 1, 3)):
        model.add(layers.Dense(
            # Tune the number of units separately
            units=hp.Int(f'units_{i}', min_value=32, max_value=512, default=32),
            activation=hp.Choice("activation", ['relu', 'tanh'])
        ))

    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation='softmax'))

    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling='log')

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        # metrics=['accuracy']
    )

    return model

In [17]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=2,
    overwrite=True,
    directory='my_dir',
    project_name='mnist_ds'
)

In [19]:
tuner.search_space_summary(extended=True)

Search space summary
Default search space size: 5
n_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': 32, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 1, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [20]:
tuner.search(X_train, y_train, epochs=2, validation_data=(X_val, y_val))

Trial 5 Complete [00h 00m 16s]
val_loss: 0.18590351194143295

Best val_loss So Far: 0.14715757220983505
Total elapsed time: 00h 01m 27s


In [33]:
tuner.oracle.get_best_trials(3)[1].summary()

Trial 4 summary
Hyperparameters:
n_layers: 3
units_0: 65
activation: tanh
dropout: True
lr: 0.00023330563491507015
units_1: 120
units_2: 100
Score: 0.18590351194143295


In [55]:
# tuner.get_best_models(num_models=1) returns a list of the best models
best_model = tuner.get_best_models(num_models=1)[0]

In [56]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 295)            │       231,575 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 377)            │       111,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 377)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         3,780 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 346,947 (1.32 MB)

 Trainable params: 346,947 (1.32 MB)

 Non-trainable params: 0 (0.00 B)

In [57]:
tuner.results_summary()

Results summary
Results in my_dir\mnist_ds
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 3 summary
Hyperparameters:
n_layers: 2
units_0: 295
activation: relu
dropout: True
lr: 0.0010153958908885
units_1: 377
units_2: 138
Score: 0.0920250341296196

Trial 0 summary
Hyperparameters:
n_layers: 3
units_0: 226
activation: tanh
dropout: False
lr: 0.00027196868413302354
units_1: 32
units_2: 32
Score: 0.1680278405547142

Trial 2 summary
Hyperparameters:
n_layers: 1
units_0: 224
activation: tanh
dropout: True
lr: 0.0006358745983798522
units_1: 230
units_2: 149
Score: 0.1743825450539589

Trial 1 summary
Hyperparameters:
n_layers: 3
units_0: 286
activation: tanh
dropout: False
lr: 0.00511185911937983
units_1: 303
units_2: 311
Score: 0.23528940975666046

Trial 4 summary
Hyperparameters:
n_layers: 3
units_0: 69
activation: tanh
dropout: True
lr: 0.009864796448312292
units_1: 299
units_2: 432
Score: 0.5155056864023209


In [77]:
grid_tuner = kt.GridSearch(
    build_model,
    objective="val_loss",
    max_trials=5,
    overwrite=True,
    directory=r'my_dir/mnist_ds',
    project_name='grid_search'
)

In [78]:
grid_tuner.search_space_summary()

Search space summary
Default search space size: 5
n_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': 32, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 1, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [80]:
grid_tuner.search(X_train, y_train, epochs=3, validation_data=(X_val, y_val))

Trial 5 Complete [00h 00m 09s]
val_loss: 0.15297657251358032

Best val_loss So Far: 0.15297657251358032
Total elapsed time: 00h 00m 43s


## Retrain the model
If you want to train the model with the entire dataset, you may retrieve the best hyperparameters and retrain the model by yourself.

In [58]:
# tuner.get_best_hyperparameters(n): returns n best hyperparameter objects
best_hps = tuner.get_best_hyperparameters(1)
model = build_model(best_hps[0])

In [59]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.3894 - val_loss: 0.1098
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0911 - val_loss: 0.0881
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0636 - val_loss: 0.0853
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0454 - val_loss: 0.1011
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0398 - val_loss: 0.0924


## Subclass HyperModel class

In [60]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            # Tune whether to shuffle the data in each epoch.
            shuffle=hp.Boolean("shuffle"),
            **kwargs
        )

In [61]:
# to check if the code works correctly

hp = kt.HyperParameters()
hypermodel = MyHyperModel()
model = hypermodel.build(hp)
hypermodel.fit(hp, model, X_train, y_train)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.4989


In [66]:
hp.get("n_layers")

1

In [69]:
# use it as a black-box optimizer for anything.
# didnt quite understand this though

In [70]:
class MyTuner(kt.RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        # Get the hp from trial.
        hp = trial.hyperparameters
        # Define "x" as a hyperparameter.
        x = hp.Float("x", min_value=-1.0, max_value=1.0)
        # Return the objective value to minimize.
        return x * x + 1


tuner = MyTuner(
    # No hypermodel or objective specified.
    max_trials=20,
    overwrite=True,
    directory="my_dir",
    project_name="tune_anything",
)

# No need to pass anything to search()
# unless you use them in run_trial().
tuner.search()
print(tuner.get_best_hyperparameters()[0].get("x"))

Trial 20 Complete [00h 00m 00s]
default_objective: 1.3918261328069328

Best default_objective So Far: 1.0001436182760177
Total elapsed time: 00h 00m 01s
-0.011984084279478724
